## [NLP FROM SCRATCH: GENERATING NAMES WITH A CHARACTER-LEVEL RNN](https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html#nlp-from-scratch-generating-names-with-a-character-level-rnn)

#### We are still hand-crafting a small RNN with a few linear layers. The big difference is instead of predicting a category after reading in all the letters of a name, we input a category and output one letter at a time. Recurrently predicting characters to form language (this could also be done with words or other higher order constructs) is often referred to as a “language model”.

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open

In [ ]:
import os
from glob import glob

In [ ]:
import string
import unicodedata

In [ ]:
home = os.environ['HOME']
data_dir = f"{home}/torch/data/"
names_dir = data_dir + "names/"

In [ ]:
all_letters = string.ascii_letters +" .,;'-"

In [ ]:
n_letters =  len(all_letters) + 1

In [ ]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [ ]:
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [ ]:
category_lines = {}
all_categories = []

In [ ]:
def findFiles(path): return glob(path)

In [ ]:
for filename in findFiles('/home/drclab/torch/data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

In [ ]:
n_categories = len(all_categories)

_____

To represent a single letter, we use a “one-hot vector” of size <1 x n_letters>. A one-hot vector is filled with 0s except for a 1 at index of the current letter, e.g. "b" = <0 1 0 0 0 ...>.

To make a word we join a bunch of those into a 2D matrix <line_length x 1 x n_letters>.

In [ ]:
def letterToIndex(letter):
    return all_letters.find(letter)

In [ ]:
import torch

In [ ]:
# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

In [ ]:
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

#### This network extends the last tutorial’s RNN with an extra argument for the category tensor, which is concatenated along with the others. The category tensor is a one-hot vector just like the letter input.

We will interpret the output as the probability of the next letter. When sampling, the most likely output letter is used as the next input letter.

I added a second linear layer o2o (after combining hidden and output) to give it more muscle to work with. There’s also a dropout layer, which randomly zeros parts of its input with a given probability (here 0.1) and is usually used to fuzz inputs to prevent overfitting. Here we’re using it towards the end of the network to purposely add some chaos and increase sampling variety.

![rnn](https://i.imgur.com/jzVrf7f.png)

In [ ]:
import torch.nn as nn

In [ ]:
class RNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(RNN, self).__init__()
        self.hidden_size =hidden_dim

        self.i2h = nn.Linear(n_categories + in_dim + hidden_dim, hidden_dim)
        self.i2o = nn.Linear(n_categories + in_dim + hidden_dim, out_dim)
        self.o2o = nn.Linear(hidden_dim + out_dim, out_dim)
        self.dropout = nn.Dropout(p=0.1)
        self.softmax = nn.LogSoftmax(dim=1)
 
    def forward(self, category,input, hidden):
        input_combo = torch.cat((category, input, hidden), dim=1)
        hidden = self.i2h(input_combo)
        out = self.i2o(input_combo)
        out_combo = torch.cat((hidden, out), 1)
        output = self.o2o(out_combo)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def init_Hidden(self):
        return torch.zeros(1, self.hidden_size)

In [34]:
import random

In [33]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

In [ ]:
category_lines

In [35]:
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

![](https://i.imgur.com/JH58tXY.png)

##### The category tensor is a one-hot tensor of size <1 x n_categories>. When training we feed it to the network at every timestep - this is a design choice, it could have been included as part of initial hidden state or some other strategy.